In [148]:
import nltk
corpus = nltk.corpus.gutenberg.sents(u'austen-emma.txt')
#corpus = corpus[:1000]
#print(corpus)

In [149]:
DELTA = 0.5 # Better to estimate using held-out
def get_unigram_counts(corpus):
    unigram_counts = {}
    for sentence in corpus:
        for word in sentence:
            word = word.lower()
            if word in unigram_counts:
                unigram_counts[word] += 1
            else:
                unigram_counts[word] = 1
    return unigram_counts
unigram_counts = get_unigram_counts(corpus)
#print(unigram_counts)

In [160]:
def get_bigram_counts(corpus):
    global total_bigram_count
    bigram_counts ={}
    for sentence in corpus:
        bigrams = [k for k in zip(sentence, sentence[1:])]
        for b in bigrams:
            w1 = b[0].lower()
            w2 = b[1].lower()
            if w1 not in bigram_counts:
                bigram_counts[w1] = {}
            if w2 not in bigram_counts[w1]:
                bigram_counts[w1][w2] = 1
            else:
                bigram_counts[w1][w2] += 1
                
    return bigram_counts
bigram_counts = get_bigram_counts(corpus)
#print(bigram_counts)

In [161]:
bigram_counts['if']

{',': 3,
 '--': 1,
 '_he_': 1,
 '_i_': 1,
 '_miss_': 1,
 '_she_': 1,
 '_we_': 2,
 '_you_': 2,
 'a': 6,
 'all': 1,
 'an': 1,
 'any': 7,
 'anywhere': 1,
 'attention': 1,
 'aunt': 1,
 'cards': 1,
 'colonel': 2,
 'compassion': 1,
 'divested': 1,
 'elton': 1,
 'emma': 4,
 'ever': 2,
 'every': 1,
 'frank': 1,
 'half': 1,
 'harriet': 5,
 'he': 70,
 'her': 2,
 'his': 5,
 'hungry': 1,
 'i': 55,
 'in': 1,
 'it': 31,
 'jane': 4,
 'left': 2,
 'meaning': 1,
 'meant': 1,
 'miss': 3,
 'mr': 10,
 'mrs': 1,
 'my': 3,
 'no': 1,
 'not': 15,
 'one': 9,
 'other': 1,
 'our': 1,
 'perry': 1,
 'poor': 2,
 'possible': 6,
 'quite': 1,
 'real': 1,
 'requisite': 1,
 'resolved': 1,
 'she': 46,
 'so': 1,
 'such': 1,
 'the': 16,
 'their': 2,
 'there': 7,
 'they': 14,
 'things': 1,
 'this': 6,
 'to': 5,
 'wanting': 1,
 'we': 13,
 'weston': 1,
 'wright': 1,
 'you': 86,
 'your': 4}

In [162]:
def get_follower_count(w, bigram_counts):
    return len(bigram_counts[w])

def get_followee_count(w, bigram_counts):
    retval = 0
    for word in bigram_counts:
        if w in bigram_counts[word]:
            retval += 1
    return retval

def get_unique_bigrams(bigram_counts):
    return sum([len(bigram_counts[key]) for key in bigram_counts])

def p_kn_lower(w, bigram_counts):
    return get_followee_count(w, bigram_counts)/get_unique_bigrams(bigram_counts)

def p_kn(ngram, unigram_counts, bigram_counts):
    w1 = ngram[0]
    w2 = ngram[1]
    prob = 0
    if w1 in bigram_counts and w2 in bigram_counts[w1]:
        prob = max(bigram_counts[w1][w2] - DELTA, 0)/unigram_counts[w1]
    
    # distributing delta to lower order ngram (here unigram)
    # KN = ABS + Productivity of the First Word * Followability/Prolificity of Second Word
    prob += (DELTA/unigram_counts[w1]) * get_follower_count(w1, bigram_counts) * p_kn_lower(w2, bigram_counts)
    return prob

In [163]:
w1 = 'of'
probs_orig = []
probs_kn = []
s = 0
for w2 in bigram_counts[w1]:
    probs_orig.append(bigram_counts[w1][w2]/unigram_counts[w1])
    probs_kn.append(p_kn((w1,w2),unigram_counts, bigram_counts))

In [164]:
sorted_probs_orig = sorted(probs_orig, reverse=True)
ranks = list(range(1, len(probs_orig)+1))
sorted_probs_kn = sorted(probs_kn, reverse=True)

In [165]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)
iplot([
    Scatter({"x" : ranks, "y": sorted_probs_orig}, name="Original"),    
    Scatter({"x" : ranks, "y": sorted_probs_kn}, name="Kneser-Ney")
])